In [1]:
import duckdb
import os
import pandas as pd
import numpy as np
import re

from duckdb.typing import *
from duckdb import DuckDBPyRelation

## Qulaity assessment

In [2]:
input_root_dir = '../data/'
output_root_dir = '../clean_data/'

if not os.path.exists(output_root_dir):
    os.mkdir(output_root_dir)

# input_files_dirs = [os.path.join(input_root_dir,'2000')]
# output_files_dirs = [os.path.join(output_root_dir,'2023')]

input_files_dirs = [os.path.join(input_root_dir,x) for x in os.listdir(input_root_dir)]
# output_files_dirs = [os.path.join(output_root_dir,x) for x in os.listdir(input_root_dir)]

In [3]:
COLUMN_NAMES = ['date','time','total_precipitation','avg_atmospheric_pressure','max_atmospheric_pressure','min_atmospheric_pressure','global_radiation','avg_air_temperature','dew_point','max_temperature','min_temperature','max_dew_point','min_dew_point','max_relative_air_humidity','min_relative_air_humidity','relative_air_humidity','wind_direction','max_wind_gust','wind_speed']
HEADER_KEYS = ['region', 'federative_unit', 'name', 'code', 'latitude', 'longitude', 'altitude', 'foundation_date']
ENCODING = 'iso-8859-1'
HEADER_SIZE = 8

In [4]:
def parse_time(time: str) -> str:
    if re.match('^([01][\d]|2[0-3])([0-5][\d]) UTC$', time) is not None:
        time = time[:2] + ':' + time[2:4]

    return time+':00'


def parse_date(date: str) -> str:
    if re.match('^[\d]{4}/[\d]{2}/[\d]{2}$', date):
        return date.replace('/', '-')
    if re.match('^[\d]{2}/[\d]{2}/[\d]{2}$', date):
        date = date.split('/')
        date[0], date[-1] = '20'+date[-1], date[0]
        return '-'.join(date)

    return date


def parse_float(value: str) -> float:
    if not re.match('[\d\-,]', value):
        return np.nan
    return float(value.replace(',', '.'))


def read_csv(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path,
                       encoding=ENCODING,
                       delimiter=';',
                       skiprows=HEADER_SIZE+1,
                       decimal=',',
                       names=COLUMN_NAMES,
                       # We have to specify `usecols`, because the files have a trailing ;, which causes an extra column to be created`
                       usecols=COLUMN_NAMES,
                       na_values=-9999,)


def exclude_null_rows(df: pd.DataFrame) -> pd.DataFrame:
    return duckdb.sql("SELECT * FROM df EXCEPT SELECT * FROM df WHERE COLUMNS(* EXCLUDE (date,time)) IS NULL").to_df()


def extract_header_values(file_path: str) -> dict:
    with open(file_path, 'r', encoding=ENCODING) as f:
        return {k:v for k, v in zip(HEADER_KEYS, [f.readline().strip().split(':;')[-1] for _ in range(HEADER_SIZE)])}


def save_to_parquet(df: pd.DataFrame, path: str) -> DuckDBPyRelation:
    return duckdb.sql("""
           SELECT concat(parse_date(date),' ',parse_time(time))::DATETIME AS datetime,
           COLUMNS(* EXCLUDE (wind_direction,max_relative_air_humidity,min_relative_air_humidity,relative_air_humidity,time,date,wind_speed)),
           max_relative_air_humidity::UTINYINT AS max_relative_air_humidity,
           min_relative_air_humidity::UTINYINT AS min_relative_air_humidity,
           relative_air_humidity::UTINYINT AS relative_air_humidity,
           wind_direction::USMALLINT AS wind_direction,
           wind_speed
           FROM df""").to_parquet(path)

In [5]:
try:
    duckdb.create_function('parse_time', parse_time, [VARCHAR], VARCHAR)
except duckdb.NotImplementedException:
    pass

try:
    duckdb.create_function('parse_date', parse_date, [VARCHAR], VARCHAR)
except duckdb.NotImplementedException:
    pass

In [6]:
stations = {}
failed_files = []

In [7]:
for input_files_dir in input_files_dirs:
    yearly_measurements = pd.DataFrame()
    for file_name in os.listdir(input_files_dir):
        file_path = os.path.join(input_files_dir,file_name)
        station_measurements = read_csv(file_path)
        station_measurements.drop_duplicates(subset=['date', 'time'])
        station_measurements = exclude_null_rows(station_measurements)
        if station_measurements.size != 0:
            file_header = extract_header_values(file_path)
            station_code = file_header['code']
            station_measurements.insert(loc=2,column='station_code',value=[station_code for _ in np.arange(len(station_measurements.index))])
            yearly_measurements = pd.concat([yearly_measurements, station_measurements])
            if stations.get(station_code) == None:
                file_header['latitude'] = parse_float(file_header['latitude'])
                file_header['longitude'] = parse_float(file_header['longitude'])
                file_header['altitude'] = parse_float(file_header['altitude'])
                file_header['foundation_date'] = parse_date(file_header['foundation_date'])
                station_code = file_header.pop('code')
                stations[station_code] = file_header
                
    save_to_parquet(yearly_measurements, os.path.join(output_root_dir,input_files_dir.split('/')[-1]+'.parquet'))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
pd.DataFrame.from_dict(stations, orient='index').reset_index(names='code').to_parquet(os.path.join(output_root_dir, 'stations.parquet'), index=False)